In [64]:
import pytest

pytestmark = [pytest.mark.unit, pytest.mark.release]

import numpy as np
from thirdai import bolt

from utils import (
    gen_single_sparse_layer_network,
    gen_numpy_training_data,
    get_categorical_acc,
    train_network,
    build_simple_distributed_bolt_network,
    train_single_node_distributed_network,
)

In [85]:
network = build_simple_distributed_bolt_network(sparsity=1.0, n_classes=10)
examples, labels = gen_numpy_training_data(n_classes=10, n_samples=1000)
train_single_node_distributed_network(
    network, examples, labels, epochs=1, update_parameters=False
)


Initializing Bolt network...
========= Bolt Network =========
InputLayer (Layer 0): dim=10
FullyConnectedLayer (Layer 1): dim=50, sparsity=1, act_func=ReLU
FullyConnectedLayer (Layer 2): dim=10, sparsity=1, act_func=Softmax
Initialized Network in 0 seconds
Distributed Network initialization done on this Node


In [86]:
gradients=network.get_weights_gradients(0)
threshold=network.get_unbiased_threshold_for_gradients(0,compression_density=0.1, sketch_biases=False)
print(threshold)
indices=network.get_unbiased_indexed_sketch_for_gradients(layer_index=0,compression_density=0.1,sketch_biases=False,seed_for_hashing=0,pregenerate_distribution=False,threshold=threshold)
print(indices)
network.set_unbiased_gradients_from_indices_values(layer_index=0,indices=indices,set_biases=False,threshold=threshold)
print(network.get_weights_gradients(0))

0.01350176427513361
[ -53 -368 -381  455  360 -283  -25  355 -122  497  409  378  105  467
 -457 -342 -434 -127 -149  207  499  384    0 -479  330    0 -475 -308
 -449 -179 -469  470 -451 -473 -313 -171  496  462 -319  353  -50  477
 -259  403 -337 -370  387  153  311  476]
[[ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.         -0.01350176
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [ 0.          0.          0.          0.          0.          0.
   0.          0.          0.          0.        ]
 [-0.01350176  0.          0.         -0.01350176  0.          0.
   0.          0.          0.          0.        ]
 [ 0.          

In [35]:
print(indices)

[0 0 0 ... 0 0 0]


In [36]:
print(indices.shape)

(5000,)
